<a href="https://colab.research.google.com/github/sebasmos/satellite.extractor/blob/main/downloader_sentinel_all_cities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install sentinelhub
! pip install sentinelhub --upgrade

     |████████████████████████████████| 211 kB 3.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 17.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 14.0 MB/s eta 0:00:01
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     |████████████████████████████████| 62 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 131 kB 22.1 MB/s eta 0:00:01
  Using cached utm-0.7.0-py3-none-any.whl
     |████████████████████████████████| 208 kB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 8.3 MB/s  eta 0:00:01
  Using cached marshmallow_enum-1.5.1-py2.py3-none-any.whl (4.2 kB)
  Using cached typing_inspect-0.7.1-py3-none-any.whl (8.4 kB)
  Created wheel for sentinelhub: filename=sentinelhub-3.6.4-py3-none-any.whl size=2319

In [2]:
!git clone https://github.com/sentinel-hub/sentinelhub-py.git

Cloning into 'sentinelhub-py'...
remote: Enumerating objects: 7174, done.
remote: Counting objects: 100% (824/824), done.
remote: Compressing objects: 100% (426/426), done.
remote: Total 7174 (delta 524), reused 593 (delta 390), pack-reused 6350
Receiving objects: 100% (7174/7174), 187.63 MiB | 25.83 MiB/s, done.
Resolving deltas: 100% (5066/5066), done.


In [1]:
# Install dependencies
! python ./sentinelhub-py/setup.py build
! python ./sentinelhub-py/setup.py install
! pip install -r sentinelhub-py/requirements.txt

running build
running install
running bdist_egg
running egg_info
creating sentinelhub.egg-info
writing sentinelhub.egg-info/PKG-INFO
writing dependency_links to sentinelhub.egg-info/dependency_links.txt
writing entry points to sentinelhub.egg-info/entry_points.txt
writing requirements to sentinelhub.egg-info/requires.txt
writing top-level names to sentinelhub.egg-info/top_level.txt
writing manifest file 'sentinelhub.egg-info/SOURCES.txt'
reading manifest file 'sentinelhub.egg-info/SOURCES.txt'
writing manifest file 'sentinelhub.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying sentinelhub.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying sentinelhub.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying sentinelhub.egg-info/dependency_links.txt -> build/bdist.linux-x

# Import functions

In [2]:
import pandas as pd
import glob
import json
import datetime as dt
from urllib.parse import unquote
from bs4 import BeautifulSoup
import os
import shutil
from sentinelhub import SHConfig
import datetime
import numpy as np
import matplotlib.pyplot as plt
import sys
from datetime import timedelta
sys.path.insert(0,'..') 
from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient, DataCollection, bbox_to_dimensions, DownloadRequest
#!pip install epiweeks
from epiweeks import Week, Year
from datetime import date
import glob, shutil

In [4]:
import sys
sys.path.append('C:/Users/Vanshi/Desktop')
from linux.request_multiple_images_linux import download_multiple_images
from linux.request_multiple_images_linux import get_folder_ID
from linux.rename_linux import get_request_individual
from src.extractor import get_images

# All cities - example 

In [46]:

# Define the range of years
img_format = "tiff"
'''
    # REAL DATA FOR COLOMBIA
    initial_year = 2015
    end_year = 2018
    years = list(range(initial_year,end_year))
    first_2015_week = 44
    end_year = 2020
    start_2015 = Week(initial_year,first_2015_week).startdate()
    weeks_2015 = list(range(first_2015_week, 53))
    weeks = list(range(1,53))
    
    # TESTING DATA
    
    
    initial_year = 2017
    end_year = 2018
    years = list(range(initial_year,end_year))
    first_2015_week = 44
    end_year = 2018
    start_2015 = Week(initial_year,first_2015_week).startdate()
    weeks_2015 = list(range(first_2015_week, 53))
    weeks = list(range(1,10))
'''
    
# REAL DATA FOR COLOMBIA
initial_year = 2015
end_year = 2018
years = list(range(initial_year,end_year+1))# +1 to consider 2018
first_2015_week = 44
#end_year = 2020
start_2015 = Week(initial_year,first_2015_week).startdate()
weeks_2015 = list(range(first_2015_week, 53))
weeks = list(range(1,53))

# path to municipalities
path_csv = "./data/coordinates720_all_municipalities.csv"
dict_coordinates =  pd.read_csv(path_csv)
dict_coordinates = dict_coordinates[25:60]
print(f"Considering batch # {len(dict_coordinates)}")
dict_coordinates = dict(zip(dict_coordinates['Municipality code'], dict_coordinates.square))
    
    
coordenadas = []
municipality = []
for k, v in dict_coordinates.items():
        municipality.append(k)
        res = [float(value) for value in v[1:-1].split(', ')]
        res = np.array(res)
        coordenadas.append(res)
        
columns_names = {'municipalities':municipality, 
                 'coordinates': coordenadas}
df = pd.DataFrame(data=columns_names)
        
new_coord = dict((columns_names))
    
# Define the range of years
img_format = "tiff"    

Considering batch # 35


In [47]:
dict_coordinates

{66170: '[-75.70969043917519, 4.8023672202627665, -75.64240255098692, 4.869845747318065]',
 73268: '[-74.91545881703425, 4.114867220354956, -74.84823432823754, 4.182345747318065]',
 95001: '[-72.67210453133762, 2.5315672205667057, -72.60498891146341, 2.5990457473180646]',
 73449: '[-74.67636117973996, 4.170167220347547, -74.60913195363875, 4.237645747318064]',
 68547: '[-73.08402129622769, 6.9548672199728, -73.01647110521445, 7.022345747318065]',
 68307: '[-73.20182742653697, 7.039367219961368, -73.13426494856803, 7.106845747318065]',
 54498: '[-73.38912036735675, 8.21236721980222, -73.32137161657373, 8.279845747318065]',
 47288: '[-74.22624251027004, 10.483467219491278, -74.158048578129, 10.550945747318064]',
 25488: '[-74.6535656646921, 4.273167220333741, -74.58632744626343, 4.340645747318064]',
 18753: '[-74.80349379842696, 2.0815672206267797, -74.73639971386835, 2.1490457473180644]',
 54405: '[-72.53878605906986, 7.800467219858206, -72.47110606767943, 7.867945747318065]',
 73408: '

Data is requested and stored in API Sentinelhub format and then it will be moved to a local folder with the correct format. 

In [48]:
root_images = "/data/"
       
temporal_path = os.path.abspath(os.getcwd()) + root_images  
    
if not os.path.isdir(temporal_path):
    os.makedirs(temporal_path)
    print(f"Creating temporal data folder in {temporal_path}")

num = len(new_coord["municipalities"])
        
print(f"Number of cities: {num}")


Number of cities: 35


In [49]:

for i in range(num):  
    print(f"City: {new_coord['municipalities'][i]} - Coordinates: {new_coord['coordinates'][i]}")	    	

    current_coor = list(new_coord["coordinates"][i])
        
    city_str = "DATASET" + "/" + str(new_coord['municipalities'][i])
    	
    if not os.path.exists(city_str):
  	  os.makedirs(city_str)
    	  
  # Download images on given range
    get_images(new_coord, current_coor, years, weeks, weeks_2015, img_format, root_images, CLIENT_ID, CLIENT_SECRET)

  # Move to structured folder in DATASETS
    root_images_store = "." + root_images
    dataset_store = "./" + city_str
    for root, dirs, files in os.walk(root_images_store, topdown=True):
 	      for name in files:
       		   path = os.path.join(root, name)
       		#print(path)
       		   if img_format in path and not dataset_store in path:
       		       shutil.copy(path, dataset_store)
    		    
images = glob.glob(dataset_store+"/*")

City: 66170 - Coordinates: [-75.70969044   4.80236722 -75.64240255   4.86984575]
Year: 2015 - week: 44
Requested week slot: 11/01/2015 - 11/08/2015 
Ended downloading
Image empty:  0
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image is being replaced..
This image 

In [ ]:
'''
root_images_store = "." + root_images
dataset_store = "./" + city_str
for root, dirs, files in os.walk(root_images_store, topdown=True):
   for name in files:
   		   path = os.path.join(root, name)
       		#print(path)
   		   if img_format in path and not dataset_store in path:
       		       shutil.copy(path, dataset_store)
    		    
images = glob.glob(dataset_store+"/*")
'''

In [ ]:
#shutil.rmtree("DATASET")
shutil.rmtree("data")

# Transfer to GCP bucket

https://philipplies.medium.com/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041 

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'mit-hst-dengue'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://colombia_sebasmos/
gs://dsrestrepo/
gs://mit-hst-dengue/


In [ ]:
bucket_name = 'colombia_sebasmos'

!gsutil -m cp -r /content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor/DATASET_final/ gs://{bucket_name}/

Copying file:///content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor/DATASET_final/76001/image_2015-11-15.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor/DATASET_final/76001/image_2015-11-01.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor/DATASET_final/76001/image_2015-11-22.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor/DATASET_final/76001/image_2015-11-29.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor/DATASET_final/76001/image_2015-11-08.tiff [Content-Type=image/tiff]...
Copying file:///content/drive/MyDrive/Dengue_prediction/Codes_download_data_sentinel/satellite.extractor/DATASET_final/7

In [ ]:
!gsutil -m cp -r \
  "gs://colombia_sebasmos/COLOMBIA_all/" \
  /DATASET/